# 1. Предобработка данных

In [ ]:
import pandas as pd

# Загрузка данных
df = pd.read_csv("MaintenanceR.csv")
df.info()

# Преобразование типов, если необходимо
# df['column'] = df['column'].astype('тип')

# 2. Исследовательский анализ данных

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Проверка на пропущенные значения
print(df.isnull().sum())

# Обнаружение и обработка выбросов
sns.boxplot(data=df.select_dtypes(include=['float64', 'int64']))
plt.xticks(rotation=90)
plt.show()

# Преобразование категориальных признаков
df_encoded = pd.get_dummies(df, drop_first=True)

# 3. Конструирование и выбор признаков

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

X = df_encoded.drop("TargetColumn", axis=1)  # замените на имя целевой переменной
y = df_encoded["TargetColumn"]

selector = SelectKBest(score_func=f_regression, k=10)
X_selected = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support()]
print("Выбранные признаки:", selected_features)

# 4. Выбор и обоснование трёх регрессоров

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

models = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(),
    "SVR": SVR()
}

# 5. Гиперпараметрическая настройка модели ML

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'RandomForest': {
        'n_estimators': [50, 100],
        'max_depth': [None, 10]
    },
    'SVR': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf']
    }
}

best_models = {}
for name in ['RandomForest', 'SVR']:
    grid = GridSearchCV(models[name], param_grid[name], cv=3, scoring='neg_mean_squared_error')
    grid.fit(X_selected, y)
    best_models[name] = grid.best_estimator_
    print(f"Best params for {name}: {grid.best_params_}")

# 6. Сравнение метрик моделей

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

def evaluate_model(model, X, y):
    mse = -cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=5).mean()
    r2 = cross_val_score(model, X, y, scoring='r2', cv=5).mean()
    return {'MSE': mse, 'R2': r2}

for name, model in best_models.items():
    metrics = evaluate_model(model, X_selected, y)
    print(f"Model: {name} — MSE: {metrics['MSE']:.4f}, R2: {metrics['R2']:.4f}")

# 7. Интерпретация и обобщающая способность

In [ ]:
# Пример текстовой интерпретации
print("\nИнтерпретация: модель RandomForest показывает высокий R2 и низкую ошибку, значит хорошо подходит для данной задачи")

# 8. Деплой: проверка на новых данных

In [ ]:
# Симулированный ввод
new_data = pd.DataFrame([{feature: 0 for feature in selected_features}])
best_model = best_models['RandomForest']
prediction = best_model.predict(new_data)
print("Прогноз для новых данных:", prediction)